In [2]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

In [3]:
import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import os
import glob
import random
import cv2
import pickle
pd.set_option('display.max_columns', None)

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

from xgboost import XGBRegressor

import torch
import torchvision
import timm
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

RANDOM_SEED = 42

def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything()

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')

Using device: cuda


In [4]:
csv_dir = '../input/petfinder-pawpularity-score'
test_dir = '../input/petfinder-pawpularity-score/test'
models_dir = '../input/pawpularity-contest-models/swin_large_patch4_window12_384_2'

test_file_path = os.path.join(csv_dir, 'test.csv')
sample_sub_file_path = os.path.join(csv_dir, 'sample_submission.csv')
print(f'Test file: {test_file_path}')
print(f'Models path: {models_dir}')

Test file: ../input/petfinder-pawpularity-score/test.csv
Models path: ../input/pawpularity-contest-models/swin_large_patch4_window12_384_2


In [5]:
test_df = pd.read_csv(test_file_path)
sample_df = pd.read_csv(sample_sub_file_path)

In [6]:
def return_filpath(name, folder):
    path = os.path.join(folder, f'{name}.jpg')
    return path

In [7]:
test_df['image_path'] = test_df['Id'].apply(lambda x: return_filpath(x, folder=test_dir))

In [8]:
test_df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,image_path
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,../input/petfinder-pawpularity-score/test/4128...
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,../input/petfinder-pawpularity-score/test/43a2...
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1,../input/petfinder-pawpularity-score/test/4e42...
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0,../input/petfinder-pawpularity-score/test/80bc...
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0,../input/petfinder-pawpularity-score/test/8f49...


In [9]:
params = {
    'model': 'swin_large_patch4_window12_384',
    'dense_features': ['Subject Focus', 'Eyes', 'Face', 'Near',
                       'Action', 'Accessory', 'Group', 'Collage',
                       'Human', 'Occlusion', 'Info', 'Blur'],
    'pretrained': False,
    'inp_channels': 3,
    'im_size': 384,
    'device': device,
    'batch_size': 16,
    'num_workers' : 2,
    'out_features': 1,
    'debug': False
}

In [10]:
if params['debug']:
    test_df = test_df.sample(frac=0.1)

# 图像增强

In [11]:
def get_test_transforms(DIM = params['im_size']):
    return albumentations.Compose(
        [
          albumentations.Resize(DIM,DIM),
          albumentations.Normalize(
              mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225],
          ),
          ToTensorV2(p=1.0)
        ]
    )

In [12]:
class CuteDataset(Dataset):
    def __init__(self, images_filepaths, dense_features, targets, transform=None):
        self.images_filepaths = images_filepaths
        self.dense_features = dense_features
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)['image']
        
        dense = self.dense_features[idx, :]
        label = torch.tensor(self.targets[idx]).float()
        return image, dense, label

In [13]:
class PetNet(nn.Module):
    def __init__(self, model_name=params['model'], out_features=params['out_features'], inp_channels=params['inp_channels'],
                 pretrained=params['pretrained'], num_dense=len(params['dense_features'])):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, 128)
        self.fc = nn.Sequential(
            nn.Linear(128 + num_dense, 64),
            nn.ReLU(),
            nn.Linear(64, out_features)
        )
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, image, dense):
        embeddings = self.model(image)
        x = self.dropout(embeddings)
        x = torch.cat([x, dense], dim=1)
        output = self.fc(x)
        return output

In [14]:
for model_name in glob.glob(models_dir + '/*.pth'):
    model = PetNet()
    model.load_state_dict(torch.load(model_name))
    model = model.to(params['device'])
    model.eval()

    test_dataset = CuteDataset(
        images_filepaths = test_df['image_path'].values,
        dense_features = test_df[params['dense_features']].values,
        targets = sample_df['Pawpularity'].values,
        transform = get_test_transforms()
    )
    test_loader = DataLoader(
        test_dataset, batch_size=params['batch_size'],
        shuffle=False, num_workers=params['num_workers'],
        pin_memory=True
    )

    temp_preds = None
    with torch.no_grad():
        for (images, dense, target) in tqdm(test_loader, desc=f'Predicting. '):
            images = images.to(params['device'], non_blocking=True)
            dense = dense.to(params['device'], non_blocking=True)
            predictions = torch.sigmoid(model(images, dense)).to('cpu').numpy()*100
            
            if temp_preds is None:
                temp_preds = predictions
            else:
                temp_preds = np.vstack((temp_preds, predictions))

    test_df[model_name.split('/')[-1].split('_')[-3]] = temp_preds

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
dense_features = []
for model_name in glob.glob(models_dir + '/*.pth'):
    dense_features.append(model_name.split('/')[-1].split('_')[-3])

# XGBoost

In [16]:
test_pred_all = None
for regressor_name in glob.glob(models_dir + '/*.bin'):
    model = XGBRegressor()
    model.load_model(regressor_name)
    test_pred = model.predict(test_df[dense_features].values)
    
    if test_pred_all is None:
        test_pred_all = test_pred
    else:
        test_pred_all += test_pred
        
test_pred_all /= (len(glob.glob(models_dir + '/*.bin')))

[20:53:50] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:53:50] WARNING: ../src/learner.cc:851: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:567: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  'Loading a native XGBoost model with Scikit-Learn interface.')


[20:53:52] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:53:52] WARNING: ../src/learner.cc:851: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility
[20:53:52] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:53:52] WARNING: ../src/learner.cc:851: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility
[20:53:52] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:53:52] WARNING: ../src/learner.cc:851: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility
[20:53:52] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:53:52] WARNING: ../src/learner.cc:851: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:567: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  'Loading a native XGBoost model with Scikit-Learn interface.')


[20:53:52] WARNING: ../src/learner.cc:851: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility
[20:53:52] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:53:52] WARNING: ../src/learner.cc:851: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility
[20:53:52] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:53:52] WARNING: ../src/learner.cc:851: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility


In [17]:
sub_df = pd.DataFrame()
sub_df['Id'] = test_df['Id']
sub_df['Pawpularity'] = test_pred_all

In [18]:
sub_df.head()

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,40.801067
1,43a2262d7738e3d420d453815151079e,40.494690
2,4e429cead1848a298432a0acad014c9d,41.704224
3,80bc3ccafcc51b66303c2c263aa38486,41.198223
4,8f49844c382931444e68dffbe20228f4,41.567101


In [19]:
sub_df.to_csv('submission.csv', index=False)